<a href="https://colab.research.google.com/github/U-11-Agar/time_series_bots/blob/testing/babuwolf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Bidirectional, LSTM, Attention, BatchNormalization, Dense,LeakyReLU,Dropout
from keras.losses import mean_squared_error
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

BASE_URL = 'https://www.alphavantage.co/query?'
API_KEY = 'K37TOUDE2WB29LQJ'

BATCH_SIZE = 120
TIME_DELAY = 20
n_steps = 20
n_features = 19  # We'll include the exchange rate and RSI as features
epoch = 2
k=0



def get_exchange_rate(BASE_URL,API_KEY,k):
    params = {
        'function': 'CURRENCY_EXCHANGE_RATE',
        'from_currency': "BTC",
        'to_currency': "USD",
        'apikey': API_KEY
    }

    try:
        return pd.DataFrame({'exchange_rate': np.random.rand()}, index=[k])
    except Exception as e:
        print(f'Error fetching forex exchange details: {e}')
        return pd.DataFrame()




def lstm_model(model,n_steps,n_features):
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, input_shape=(n_steps, n_features), use_bias=True,activation="linear")))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.1111),dropout=0.10)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.20)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.2222),dropout=0.20)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.30)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.3333),dropout=0.30)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.40)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.4444),dropout=0.40)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.5555)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.5555),dropout=0.50)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.5555)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.6666),dropout=0.40)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.40)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.7777),dropout=0.30)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.30)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.8888),dropout=0.20)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.20)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.9999),dropout=0.10)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, use_bias=True,activation='linear',dropout=0.05)))
  model.add(Dense(units=n_steps, activation='linear'))
  model.compile(loss='mean_absolute_error', optimizer=Adam(0.5,clipnorm=0.000000051), metrics=['mse'], run_eagerly=True)
  return model


def splitsequence(data, n_steps, n_features):
    X, y = [], []
    for i in range(n_steps, data.shape[0]-n_steps):
        X.append(data.iloc[i-n_steps:i, :].values)
        y.append(data.iloc[i:i+n_steps, [0]].values)
    X, y = np.array(X), np.array(y)
    X = X.reshape(X.shape[0], n_steps, n_features)
    y = y.reshape(y.shape[0], n_steps, 1)
    return X, y

def plot_predictions(predictions,data):
    plt.figure(figsize=(10, 7))
    plt.plot(data, color='red', label="exchange_rate")
    plt.plot(np.arange(len(data)-1, -1+len(data) + len(predictions)), predictions, color='green', label='future')
    plt.legend()
    plt.show()



def create_data_model(df):
  data_model = pd.DataFrame(df['exchange_rate'],index=df.index,columns=["exchange_rate"])
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],2)],axis=1)
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],3)],axis=1)
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],5)],axis=1)
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],8)],axis=1)
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],13)],axis=1)
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],21)],axis=1)
  data_model = pd.concat([data_model,calculate_relative_strength_index(df['exchange_rate'],9)],axis=1)
  data_model = pd.concat([data_model,calculate_relative_strength_index(df['exchange_rate'],14)],axis=1)
  data_model = pd.concat([data_model,calculate_bollinger_bands(df['exchange_rate'],10)],axis=1)
  data_model = pd.concat([data_model,calculate_bollinger_bands(df['exchange_rate'],20)],axis=1)
  data_model = pd.concat([data_model,calculate_stochastic_oscillator(df['exchange_rate'],9)],axis=1)
  data_model = pd.concat([data_model,calculate_stochastic_oscillator(df['exchange_rate'],14)],axis=1)
  data_model.dropna(inplace=True,axis=0)
  return data_model
    # print(indicators_df.head())

def calculate_moving_average(dataframe, win):
    ma = dataframe.rolling(window=win).mean().dropna(axis=0)
    return pd.Series(ma, name=f'MA_{win}')

def calculate_relative_strength_index(dataframe, window):
    delta = dataframe.diff()
    up = delta.where(delta > 0, 0)
    down = -delta.where(delta < 0, 0)
    ema_up = up.ewm(span=window, adjust=True,ignore_na=True).mean().dropna(axis=0)
    ema_down = down.ewm(span=window, adjust=True,ignore_na=True).mean().dropna(axis=0)
    avg_gain = up.rolling(window=window, min_periods=1).mean().dropna(axis=0)
    avg_loss = down.rolling(window=window, min_periods=1).mean().dropna(axis=0)
    rs_mva = avg_gain / avg_loss
    rs_ewm = ema_up / ema_down
    rsi_mva = 100 - (100 / (1 + rs_mva))
    rsi_ewm = 100 - (100 / (1 + rs_ewm))
    return pd.DataFrame({f"RSI_ewm_{window}": rsi_ewm,f"RSI_mva_{window}":rsi_mva},index=dataframe.index)

def calculate_bollinger_bands(dataframe, window):
    ma = dataframe.rolling(window=window).mean().dropna(axis=0)
    std = dataframe.rolling(window=window).std().dropna(axis=0)
    upper_band = ma + (2 * std)
    lower_band = ma - (2 * std)
    return pd.DataFrame({f'bb_Upper_Band_{window}': upper_band, f'bb_Lower_Band_{window}': lower_band},index=dataframe.index)

def calculate_stochastic_oscillator(dataframe, window):
    low = dataframe.rolling(window=window).min().dropna(axis=0)
    high = dataframe.rolling(window=window).max().dropna(axis=0)
    k = 100 * ((dataframe - low) / (high - low))
    d = k.rolling(window=3).mean()
    return pd.DataFrame({f'%K{window}': k, f'%D{window}': d},index=dataframe.index)

In [ ]:
model = Sequential()
model = lstm_model(model,n_steps,n_features)
df = pd.DataFrame(columns=['exchange_rate'])
scaler_1=MinMaxScaler()
scaler_2=StandardScaler()
x = [0] * n_steps
predictions_df = pd.DataFrame([x])
k=1
while k:
    k+=1
    forex_exchange = get_exchange_rate(BASE_URL,API_KEY,k)
    if not forex_exchange.empty:
        df = pd.concat([df, forex_exchange], axis=0).drop_duplicates()
        # df = df[~df.index.duplicated(keep='last')]  # Drop duplicate indices
        num_samples_1 = df.shape[0]
        if(num_samples_1>=141):
          data_model = create_data_model(df)
          df_scaled_1=pd.DataFrame(scaler_1.fit_transform(data_model),columns=data_model.columns)
          df_scaled_2=pd.DataFrame(scaler_2.fit_transform(df_scaled_1),columns=data_model.columns)
          print(df_scaled_2.shape)
          num_samples_2 = df_scaled_2.shape[0]
          if num_samples_2 >= (BATCH_SIZE):
              start_index = num_samples_2 - (BATCH_SIZE)
              features, labels = splitsequence(df_scaled_2.iloc[start_index:, :], n_steps, n_features)
              print(features, labels)
              for _ in range(epoch):
                  model.train_on_batch(features, labels)
              x_ = df_scaled_2.iloc[num_samples_2-n_steps:,:].values.reshape(1, n_steps, n_features)
              predictions = np.array(model.predict(x_))
              print(predictions[0])
              predictions_df = predictions_df.append(pd.DataFrame(predictions,columns=predictions_df.columns), ignore_index=True)
              if num_samples_2 % 5 == 0:
                plt.plot(predictions_df.tail(20))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
df = pd.DataFrame(columns=['exchange_rate'])
def get_exchange_rate(k, noise_level=np.random.randn(), trend=np.random.randn()):
    # Assuming you want to generate synthetic exchange rate data using a sine wave
    # with some added noise and trend.
    # k: index for generating new data point
    # noise_level: magnitude of noise to be added
    # trend: linear trend added to the data

    # Frequency of the sine wave (in cycles per unit time)
    freq = 0.1

    # Amplitude of the sine wave
    amplitude = 100

    # Generate time series data using sine wave
    exchange_rate = amplitude * np.sin(freq * k) + trend * k

    # Add noise to the data
    exchange_rate += noise_level * np.random.randn()

    # Return the synthetic exchange rate data
    return pd.DataFrame({'exchange_rate': exchange_rate}, index=[k])

# Example usage:
for k in range(100):
    forex_exchange = get_exchange_rate(k)
    if not forex_exchange.empty:
        df = pd.concat([df, forex_exchange], axis=0).drop_duplicates()
print(df)
plt.plot(df)


In [ ]:
np.random.rand()

In [ ]:
np.linspace(0, 10, 1000)

In [ ]:
x = [0]*n_steps
[x]

In [ ]:
k+=1
forex_exchange = get_exchange_rate(BASE_URL,API_KEY,k)
if not forex_exchange.empty:
    df = pd.concat([df, forex_exchange], axis=0).drop_duplicates()
print(df)

In [ ]:
import requests
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Input, Bidirectional, LSTM, Attention, BatchNormalization, Dense,LeakyReLU,Dropout
from keras.losses import mean_squared_error
from keras.metrics import RootMeanSquaredError
# from keras.initializers import RandomUniform
from keras.optimizers import Adam
from keras.models import Sequential
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

BASE_URL = 'https://www.alphavantage.co/query?'
API_KEY = 'K37TOUDE2WB29LQJ'

BATCH_SIZE = 120
TIME_DELAY = 20
n_steps = 20
n_features = 19  # We'll include the exchange rate and RSI as features
epoch = 2
k=0
import pandas as pd
def get_exchange_rate(BASE_URL,API_KEY,k):
    params = {
        'function': 'CURRENCY_EXCHANGE_RATE',
        'from_currency': "BTC",
        'to_currency': "USD",
        'apikey': API_KEY
    }

    try:
        # response = requests.get(BASE_URL, params=params)
        # data = response.json()
        # k+=1
        # x=data['Realtime Currency Exchange Rate']
        # return pd.DataFrame({'exchange_rate':np.random.rand()},index = [k])
        # return pd.DataFrame({'exchange_rate':x['5. Exchange Rate']},index = [pd.to_datetime(x['6. Last Refreshed'])])
        return pd.DataFrame({'exchange_rate': np.random.rand()}, index=[k])
    except Exception as e:
        print(f'Error fetching forex exchange details: {e}')
        return pd.DataFrame()




def lstm_model(model,n_steps,n_features):
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, input_shape=(n_steps, n_features), use_bias=True,activation="linear")))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.1111),dropout=0.10)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.2222),dropout=0.20)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.3333),dropout=0.30)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.4444),dropout=0.40)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.5555),dropout=0.50)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.6666),dropout=0.40)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.7777),dropout=0.30)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.8888),dropout=0.20)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation=LeakyReLU(alpha=0.9999),dropout=0.10)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, return_sequences=True, use_bias=True,activation='tanh',dropout=0.05)))
  model.add(Bidirectional(LSTM(units=BATCH_SIZE, use_bias=True,activation='linear',dropout=0.05)))
  model.add(Dense(units=n_steps, activation='linear'))
  # model.compile(loss='Binary_cr', optimizer=Adam(0.001,clipnorm=0.01), metrics=['accuracy','mse','mae'])
  model.compile(loss='mean_absolute_error', optimizer=Adam(0.05,clipnorm=0.00000051), metrics=['mse'], run_eagerly=True)
  # model.compile(optimizer='adam', loss=mean_squared_error)
  return model


def splitsequence(data, n_steps, n_features):
    # The function to split data into sequences for LSTM training remains the same
    X = []
    y = []
    for i in np.arange(n_steps, (len(data)-n_steps)):
        X.append(data.iloc[i-n_steps:i, :].values)
        y.append(data.loc[i:i+(n_steps-1),["exchange_rate"]].values)
    X, y = np.array(X), np.array(y)
    X = X.reshape(X.shape[0], n_steps, n_features)
    y = y.reshape(y.shape[0], n_steps, 1)
    return X, y

def plot_predictions(predictions,data):
    # plt.figure(figsize=(20, 10))
    # # plt.plot(predictions_df['real_value'], label='Real Value')
    # # plt.show()
    # plt.plot(data,color='red',lable="exchange_rate")
    # plt.plot(np.array(data)+predictions_df,color='green',lable='future')
    # plt.show()
    # # plt.xlabel('Time')
    # # plt.ylabel('Exchange Rate')
    # # plt.title('Real vs Predicted Exchange Rate')
    # # plt.legend()
    plt.figure(figsize=(10, 7))
    plt.plot(data, color='red', label="exchange_rate")
    plt.plot(np.arange(len(data)-1, -1+len(data) + len(predictions)), predictions, color='green', label='future')
    plt.legend()
    plt.show()



def create_data_model(df):
  data_model = pd.DataFrame(df['exchange_rate'],index=df.index,columns=["exchange_rate"])
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],2)],axis=1)
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],3)],axis=1)
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],5)],axis=1)
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],8)],axis=1)
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],13)],axis=1)
  data_model = pd.concat([data_model,calculate_moving_average(df['exchange_rate'],21)],axis=1)
  data_model = pd.concat([data_model,calculate_relative_strength_index(df['exchange_rate'],9)],axis=1)
  data_model = pd.concat([data_model,calculate_relative_strength_index(df['exchange_rate'],14)],axis=1)
  data_model = pd.concat([data_model,calculate_bollinger_bands(df['exchange_rate'],10)],axis=1)
  data_model = pd.concat([data_model,calculate_bollinger_bands(df['exchange_rate'],20)],axis=1)
  data_model = pd.concat([data_model,calculate_stochastic_oscillator(df['exchange_rate'],9)],axis=1)
  data_model = pd.concat([data_model,calculate_stochastic_oscillator(df['exchange_rate'],14)],axis=1)
  data_model.dropna(inplace=True,axis=0)
  return data_model
    # print(indicators_df.head())


def calculate_moving_average(dataframe, win):
    ma = dataframe.rolling(window=win).mean().dropna(axis=0)
    return pd.Series(ma, name=f'MA_{win}')


def calculate_relative_strength_index(dataframe, window):
    delta = dataframe.diff()
    up = delta.where(delta > 0, 0)
    down = -delta.where(delta < 0, 0)
    ema_up = up.ewm(span=window, adjust=True,ignore_na=True).mean().dropna(axis=0)
    ema_down = down.ewm(span=window, adjust=True,ignore_na=True).mean().dropna(axis=0)
    avg_gain = up.rolling(window=window, min_periods=1).mean().dropna(axis=0)
    avg_loss = down.rolling(window=window, min_periods=1).mean().dropna(axis=0)
    rs_mva = avg_gain / avg_loss
    rs_ewm = ema_up / ema_down
    rsi_mva = 100 - (100 / (1 + rs_mva))
    rsi_ewm = 100 - (100 / (1 + rs_ewm))
    return pd.DataFrame({f"RSI_ewm_{window}": rsi_ewm,f"RSI_mva_{window}":rsi_mva},index=dataframe.index)

def calculate_bollinger_bands(dataframe, window):
    ma = dataframe.rolling(window=window).mean().dropna(axis=0)
    std = dataframe.rolling(window=window).std().dropna(axis=0)
    upper_band = ma + (2 * std)
    lower_band = ma - (2 * std)
    return pd.DataFrame({f'bb_Upper_Band_{window}': upper_band, f'bb_Lower_Band_{window}': lower_band},index=dataframe.index)

def calculate_stochastic_oscillator(dataframe, window):
    low = dataframe.rolling(window=window).min().dropna(axis=0)
    high = dataframe.rolling(window=window).max().dropna(axis=0)
    k = 100 * ((dataframe - low) / (high - low))
    d = k.rolling(window=3).mean()
    return pd.DataFrame({f'%K{window}': k, f'%D{window}': d},index=dataframe.index)







In [ ]:



model = Sequential()
model = lstm_model(model,n_steps,n_features)
df = pd.DataFrame(columns=['exchange_rate'])
scaler=MinMaxScaler()
# scaler=StandardScaler()
x=[i for i in range(n_steps)]
predictions_df = pd.DataFrame([x])
k=1
while k:
    k+=1
    forex_exchange = get_exchange_rate(BASE_URL,API_KEY,k)
    if not forex_exchange.empty:
        df = pd.concat([df, forex_exchange], axis=0).drop_duplicates()
        df = df[~df.index.duplicated(keep='last')]  # Drop duplicate indices
        num_samples_1 = df.shape[0]
        if(num_samples_1>=140):
          data_model = create_data_model(df)
          df_scaled_model=pd.DataFrame(scaler.fit_transform(data_model),columns=data_model.columns)
          print(df_scaled_model.shape)
          num_samples_2 = df_scaled_model.shape[0]
          if num_samples_2 >= (BATCH_SIZE):
              start_index = num_samples_2 - (BATCH_SIZE)

              # if predictions_df.shape[0] != 0:
              #     predictions_df.iloc[-1, 0] = df_scaled_model['exchange_rate'].iloc[-2]
              #     predictions_df.iloc[-1, 2] = abs(df_scaled_model['exchange_rate'].iloc[-2] - predictions_df.iloc[-1, 1])


              features, labels = splitsequence(df_scaled_model.iloc[start_index:, :], n_steps, n_features)

              for _ in range(epoch):
                  model.train_on_batch(features, labels)

              x_ = df_scaled_model.iloc[num_samples_2-n_steps:,:].values.reshape(1, n_steps, n_features)
              predictions = np.array(model.predict(x_))
              print(predictions[0])

              # plot_predictions(predictions[0],df_scaled_model['exchange_rate'])
              # plot_predictions(predictions[0], df_scaled_model['exchange_rate'])


              # new_predictions = {
              #     # 'real_value': np.nan,
              #     'predicted_value': predictions,
              #     # 'absolute_error': np.nan,
              #     # 'rsi': calculate_relative_strength_index(pd.DataFrame(predictions), 5)
              # }

              predictions_df = predictions_df.append(pd.DataFrame(predictions,columns=predictions_df.columns), ignore_index=True)
              if num_samples_2 % 5 == 4:
                plt.plot(predictions_df.tail(20))
              # print(predictions_df)

              # # Use the LSTM predictions and RSI for buy/sell decisions
              # if len(predictions_df) > n_steps:
              #     # Assuming the LSTM predicts price increase and RSI is oversold, we'll buy, and vice versa
              #     if (predictions_df.iloc[-1]['predicted_value'] > predictions_df.iloc[-2]['predicted_value']) and (predictions_df.iloc[-1]['rsi'] < 30):
              #         print(f"Buying at {df.index[-1]}")
              #         # Place buy order using your trading API
              #         # r.orders.order_buy_market(SYMBOL, 1, timeInForce='gfd')
              #     elif (predictions_df.iloc[-1]['predicted_value'] < predictions_df.iloc[-2]['predicted_value']) and (predictions_df.iloc[-1]['rsi'] > 70):
              #         print(f"Selling at {df.index[-1]}")
              #         # Place sell order using your trading API
              #         # r.orders.order_sell_market(SYMBOL, 1, timeInForce='gfd')

              # time.sleep(0.33)
    # else:
        # time.sleep(0.33)


In [ ]:
print(features.shape, labels.shape)

In [ ]:
df_scaled_model.describe()

In [ ]:
x=[i for i in range(n_steps)]
predictions_df = pd.DataFrame([x])
predictions_df = predictions_df.append(pd.DataFrame([x],columns=predictions_df.columns), ignore_index=True)
print(predictions_df)

In [ ]:
plt.plot(np.array(df_scaled_model['exchange_rate']).reshape(df_scaled_model.shape[0],1))

In [ ]:
np.arange(len(df_scaled_model['exchange_rate']), len(df_scaled_model['exchange_rate']) + len(predictions))

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_predictions(predictions, data):
    plt.figure(figsize=(20, 10))
    plt.plot(data, color='red', label="exchange_rate")
    plt.plot(np.arange(len(data)-1, -1+len(data) + len(predictions)), predictions, color='green', label='future')
    plt.legend()
    plt.show()

# Assuming you have 'predictions' and 'df_scaled_model['exchange_rate']'
# Replace these with your actual data
predictions = [[10, 15, 12, 8, 6]]  # Example data for predictions
df_scaled_model = pd.DataFrame({'exchange_rate': [5, 8, 10, 9, 11]})  # Example data for exchange_rate

plot_predictions(predictions[0], df_scaled_model['exchange_rate'])


In [ ]:
def plot_predictions(predictions,data):
    plt.figure(figsize=(20, 10))
    # plt.plot(predictions_df['real_value'], label='Real Value')
    # plt.show()
    plt.plot(data,color='red',label="exchange_rate")
    plt.plot(plt.plot(np.array(data).reshape(data.shape[0],1))+predictions,color='green',label='future')
    plt.show()
    # plt.xlabe
plot_predictions(predictions[0],df_scaled_model['exchange_rate'])

In [ ]:
predictions

In [ ]:
np.array(data).reshape(1,data.shape[0])+[predictions]

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have two arrays 'x' and 'y'
x = [1, 2, 3, 4, 5]
y = [10, 15, 12, 8, 6]

# Plot the 'x' array first
plt.plot(x+y, label='X')

# Plot the 'y' array after the 'x' array
# plt.plot(y, label='Y')  # This plot is used for creating the label only

# Show the legend
plt.legend()

# Show the plot
plt.show()


In [ ]:
x=df_scaled_model.loc[:,["exchange_rate"]].values
y=prediction
# plt.plot(df_scaled_model.loc[:,["exchange_rate"]].values)

In [ ]:
print(features.shape, labels.shape)

plt.plot(predictions[0])

In [ ]:
predictions_df = predictions_df.append({"predicted_value":predictions.reshape(n_steps,1)}, ignore_index=True)
print(predictions_df)

In [ ]:
x=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,4,5,6,7,8,9]
df=pd.DataFrame([x],)
df

In [ ]:
df=pd.DataFrame()
for i in range(100):
  forex_exchange=x(i)
  df = pd.concat([df, forex_exchange],axis=0).drop_duplicates()
  df.index.drop_duplicates()
  num_samples_1 = df.shape[0]
  print(num_samples_1)
print(df)
data=df
X = []
y = []
for i in np.arange(20, (len(data)-20)):
    X.append(data.iloc[i-20:i, :].values)
    y.append(data.loc[i:i+20,["exchange_rate"]].values)
X, y = np.array(X), np.array(y)
X = X.reshape(X.shape[0], n_steps, n_features)
# y = y.reshape(y.shape[0], n_steps, 1)
print(X,y)

In [ ]:
np.arange(20,len(data)-20)

In [ ]:
data.loc[0:20,["exchange_rate"]]

In [ ]:
X=[]
y=[]
for i in np.arange(n_steps, (len(data)-n_steps+1)):
    X.append(data.iloc[i-n_steps:i, :].values)
    y.append(data.loc[i:i+(n_steps-1),["exchange_rate"]].values)

In [ ]:
df_scaled_model.shape

In [ ]:
k=[1]
k+=[1]
print(k)

In [ ]:
def x(i):
  # for i in range(10):
    # i+=1
    # x=data['Realtime Currency Exchange Rate']
    return pd.DataFrame({'exchange_rate':np.random.rand()},index = [i])

In [ ]:
for i in range(100):
  forex_exchange=x(i)
  df = pd.concat([df, forex_exchange],axis=0).drop_duplicates()
  df.index.drop_duplicates()
  num_samples_1 = df.shape[0]
  print(num_samples_1)
print(df)

In [ ]:
import pandas as pd
import numpy as np

def x(i):
    return pd.DataFrame({'exchange_rate': np.random.rand()}, index=[i])

# Initialize an empty DataFrame
df = pd.DataFrame()

num_samples_list = []

for i in range(10):
    forex_exchange = x(i)
    df = pd.concat([df, forex_exchange], axis=0).drop_duplicates()
    df = df[~df.index.duplicated(keep='last')]  # Drop duplicate indices
    num_samples = df.shape[0]
    num_samples_list.append(num_samples)
    # print(num_samples)

print(df)
print(num_samples_list)



In [ ]:
for i in np.arange(10, 20):
  print(i)

In [ ]:
df.iloc[0:6]